# Topic 2: Uncertainty

## Associated Reading: Bishop 1.2

# 1. Overfitting
This week, we'll continue our analysis of the simple infant mortality rate that we have been working with.  Once again, we will read the data from file, drop outliers, and normalize.  However, this time, let's imagine that rather than just predicting the IMR for Venezuela, we'll try to make predictions for 1/2 of the countries on the list.  Thus, we're going to split our data into two sets: a *training set*, which we will use to adjust the parameters of (or train) our model, and a *test set*, which we will use after training to determine how well our model is at predicting things it hasn't seen before.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(17)

data = pd.read_csv('datasets/birthrate.dat',header=0,sep=r"\s{2,}",engine='python',index_col=0)
data = data.drop('United States')

pci_min = data['PCI'].min()
pci_range = data['PCI'].max() - pci_min
x = (data['PCI'] - pci_min)/pci_range

imr_min = data['Infant Mortality'].min()
imr_range = data['Infant Mortality'].max() - imr_min
y = (data['Infant Mortality'] - imr_min)/imr_range

m = len(x)
random_indices = np.random.permutation(range(m))
train_indices = random_indices[:m//2]
test_indices = random_indices[m//2:]

x_train = x[train_indices]
y_train = y[train_indices]

x_test = x[test_indices]
y_test = y[test_indices]

plt.plot(x_train,y_train,'ro')
plt.plot(x_test,y_test,'bo')
plt.xlabel('Normalized PCI')
plt.ylabel('Normalized IMR')
plt.show()

Note that our purpose from the last exercise has changed a bit: rather than predict the value of IMR for a single country, now we want to ask whether, in general, our model is good at predicting *any* country's IMR.  

In the last unit's in-class assignment, we developed some code that would allow us to fit polynomials of various complexity to these observations.  We also found that for models with many degrees of freedom (lots of parameters relative to the data), that some of the resulting fits were pretty wild.

In [ ]:
def fit_poly(x,y,d):
    X = np.vander(x,d,increasing=True)
    w = np.linalg.solve(X.T@X,X.T@y)
    y_pred = X @ w
    return y_pred,w

def evaluate_poly(x,w):
    X = np.vander(x,len(w),increasing=True)
    return X @ w

def get_smooth_prediction(w,xmin=0,xmax=1):
    x = np.linspace(xmin,xmax,101)
    y = evaluate_poly(x,w)
    return x,y

d = len(x_train)

y_pred_train,w = fit_poly(x_train,y_train,d)
x_plotting,y_plotting = get_smooth_prediction(w)

plt.plot(x_train,y_train,'ro')
plt.plot(x_plotting,y_plotting,'k-')
plt.xlabel('Normalized PCI')
plt.ylabel('Normalized IMR')
plt.show()

Note that we're getting almost a perfect fit to the training data.  **But do we believe the predictions that these complicated models produce?** Let's come up with a strategy to find out.  How shall we quantify how well our model fits the training data?    

In [ ]:
def get_mse(y_pred,y_obs):
    return 1./len(y_pred)*sum((y_pred-y_obs)**2)

mse_train = []
w_trained = []

for d in range(1,12):
    y_train_pred,w = fit_poly(x_train,y_train,d)
    mse_train.append(get_mse(y_train_pred,y_train))
    w_trained.append(w)
    
plt.plot(range(0,11),mse_train)
plt.xlabel('Polynomial Degree')
plt.ylabel('MSE')
plt.show()

This is great.  We get what amounts to zero error as the complexity of the model increases. 

But what about the error when it comes to predicting stuff that we didn't use to train.  We can evaluate that using the *test set* that we split off from the main dataset earlier.  In fact, we can use the same metric to quantify it.

In [ ]:
mse_test = []

for d,w in zip(range(1,12),w_trained):
    y_test_pred = evaluate_poly(x_test,w)
    mse_test.append(get_mse(y_test_pred,y_test))
    
plt.plot(range(0,11),mse_test)
plt.xlabel('Polynomial Degree')
plt.ylabel('MSE')
plt.show()

What if we plot both the training and test error on top of one another?

In [ ]:
plt.plot(range(0,11),mse_train,'r-')
plt.plot(range(0,11),mse_test,'b-')
plt.xlabel('Polynomial Degree')
plt.ylabel('MSE')
plt.show()

The model's training error continues to decline as we add more *complexity* to the model.  However, **based on the test error, is there any advantage to using a more complex model?  at what point does using a more complex model cease to be an advantage?**

Perhaps a more critical question is as follows: **What is going wrong when we use a more complex model?**

## 2.  Observational uncertainty
Taking measurements is hard.  There are many ways in which data may *not perfectly quantify the phenomenon it is supposed to be quantifying*.  Let's take the IMR, for example: there might be cultural biases against reporting or government statistics might be corrupted. This is called *observational uncertainty*.  (More insidiously, it is always the case that the models that we use are incorrect, though some may still be useful.  This latter reason behind why data diverge from predictions is called *structural uncertainty*, however we will ignore this for the time being).  We need our models to be robust to this kind of observational uncertainty: our fitting procedure should, in some sense, be able to ignore these observational errors and still fit the underlying quantity that the observations are imperfectly representing.  Let's experiment with how robust our polynomial models are to observational error.  

The code I provide below simulates another reality.  What do I mean by that?  Let's imagine for a moment that the error in our measurements is *random*, in the sense that our observation is just the true value of IMR corrupted by some random number.  What this code does is (for our purposes) to provide the same observations, but with that random number something different.     

In [ ]:
def get_new_y_train(sigma=0.02):
    return y_train + sigma*np.random.randn(len(y_train))

plt.plot(x_train,get_new_y_train(),'o')
plt.plot(x_train,get_new_y_train(),'o')
plt.plot(x_train,get_new_y_train(),'o')
plt.xlabel('Relative PCI')
plt.ylabel('Relative IMR')
plt.show()

Each of the above colors represents a dataset which (very loosely speaking) is the dataset we might have collected in a different reality.  Now we can ask the question, how robust is the process of fitting a line to these errors?

In [ ]:
ws = []

for i in range(1000):
    y_train_new = get_new_y_train(sigma=0.05)
    y_train_pred,w = fit_poly(x_train,y_train_new,2)
    ws.append(w)
    x_plotting,y_plotting = get_smooth_prediction(w)
    plt.plot(x_train,y_train_new,'r.',alpha=0.3)
    plt.plot(x_plotting,y_plotting,'k-',alpha=0.3)
plt.xlim(0,1)
plt.ylim(0,1)
plt.show()